In [1]:
using PMP, CSV, DataFrames, Distributions, Extremes, Gadfly, Dates#, Cairo, Fontconfig

We want to estimate stations PMP from usual methods:
* Humidity maximization with $PW_{max}$;
* Humidity maximization with $PW_{100}$;
* Hershfield empirical method with K=15.

## Data download
### Precipitations
#### Montréal

In [2]:
data_mtl = CSV.read("rain-mtl.csv", DataFrame);
first(data_mtl, 5)

Row,Date,Rain
,Date,Float64
1,1953-05-01,11.4
2,1953-05-04,2.0
3,1953-05-11,0.5
4,1953-05-12,3.0
5,1953-05-13,3.3


#### St-Hubert

In [3]:
data_sh = CSV.read("rain-sh.csv", DataFrame);
first(data_sh, 5)

Row,Date,Rain
,Date,Float64
1,1949-07-04,12.4
2,1949-07-09,27.2
3,1949-07-10,0.5
4,1949-07-17,1.3
5,1949-07-18,2.8


### Dewpoint
For humidity maximization, we need hourly dewpoint data for Montréal and St-Hubert

#### Montréal

In [76]:
mtl_dew_hour = CSV.read("dewpoint-mtl.csv")
first(mtl_dew_hour, 5)

Row,DateTime,Dew
,DateTime,Float64
1,1953-05-01T00:00:00,0.0
2,1953-05-01T00:00:00,0.0
3,1953-05-01T00:00:00,-0.6
4,1953-05-01T00:00:00,1.1
5,1953-05-01T00:00:00,0.6


#### St-Hubert

In [75]:
sh_dew_hour = CSV.read("dewpoint-sh.csv")
first(sh_dew_hour, 5)

Row,DateTime,Dew
,DateTime,Float64
1,1953-05-01T00:00:00,-1.1
2,1953-05-01T00:00:00,-1.1
3,1953-05-01T00:00:00,-0.6
4,1953-05-01T00:00:00,-1.1
5,1953-05-01T00:00:00,-0.6


---
# Montréal
## Humidity maximization
#### Storm selection : annual greatest 10% storms

In [96]:
p = 0.1  #10%
storm_mtl = storm_selection(data_mtl.Rain, data_mtl.Date, p)
first(storm_mtl, 5)

Row,Date,Rain
,Date,Float64
1,1953-08-04,39.6
2,1953-06-08,32.5
3,1953-07-06,31.8
4,1953-05-14,21.3
5,1953-05-17,19.8


#### Precipitable water (PW)

In [97]:
obs_freq = 1     # One observation per hour
time_int = 12    # Keep the smallest observation on each moving window of size 12h

# Calculate appropriate pw for each storm
pw_storm_mtl = PW_storm(storm_mtl.Date, mtl_dew_hour.Dew, mtl_dew_hour.DateTime, 24, obs_freq, time_int)
first(pw_storm_mtl, 5)

Row,Date,Dew,PW
,DateTime,Float64,Float64
1,1953-08-04T00:00:00,12.8,27.4
2,1953-06-08T00:00:00,10.0,21.0
3,1953-07-06T00:00:00,18.9,47.6
4,1953-05-14T00:00:00,6.7,15.7
5,1953-05-17T00:00:00,10.6,22.2


#### Maximization ratio

In [98]:
# PW dataset construction
pw_mtl = DataFrame(Date = mtl_dew_hour.DateTime)
pw_mtl.PW = dewpoint_to_PW.(mtl_dew_hour.Dew)

pw_max_mtl = PW_max(pw_mtl.PW, pw_mtl.Date)[2:7, :]                  # PW max for months of interest
pw_100_mtl = PW_return_period(pw_mtl.PW, pw_mtl.Date, 100)[2:7, :]   # PW 100 fot months of interest

Row,Month,PW_rp
,Int64,Float64
1,5,63.3157
2,6,81.5128
3,7,85.7006
4,8,92.3363
5,9,79.1104
6,10,57.9572


#### PMP

In [99]:
pmp_max_mtl = PMP_mm(storm_mtl.Rain, pw_storm_mtl.PW, storm_mtl.Date, pw_max_mtl.PW_max)

282.1923076923077

In [100]:
pmp_100_mtl = PMP_mm(storm_mtl.Rain, pw_storm_mtl.PW, storm_mtl.Date, pw_100_mtl.PW_rp)

284.43588001071834

## Hershfield K=15

In [16]:
mtl_pmp_hershfield = PMP_Hershfield(data_mtl.Rain, data_mtl.Date, 15)

260.760800470233


---
# St-Hubert
## Humidity maximization
For St-Hubert, we only have dewpoint from 1953. We have to cut rain data.

In [111]:
data_sh_short = data_sh[233:end, :];

#### Storm selection : Storm selection : annual greatest 10% storms

In [112]:
p = 0.1  #10%
storm_sh = storm_selection(data_sh_short.Rain, data_sh_short.Date, p)
first(storm_sh, 5)

Row,Date,Rain
,Date,Float64
1,1953-07-06,39.9
2,1953-08-04,27.9
3,1953-06-08,26.4
4,1953-06-25,24.4
5,1953-05-17,22.6


#### Precipitable water

In [113]:
obs_freq = 1     # One observation per hour
time_int = 12    # Keep the smallest observation on each moving window of size 12h

# Calculate appropriate pw for each storm
pw_storm_sh = PW_storm(storm_sh.Date, sh_dew_hour.Dew, sh_dew_hour.DateTime, 24, obs_freq, time_int)
first(pw_storm_sh, 5)

Row,Date,Dew,PW
,DateTime,Float64,Float64
1,1953-07-06T00:00:00,18.3,45.2
2,1953-08-04T00:00:00,12.2,25.6
3,1953-06-08T00:00:00,9.4,19.8
4,1953-06-25T00:00:00,6.7,15.7
5,1953-05-17T00:00:00,11.1,23.2


#### Maximization ratio

In [114]:
# PW dataset construction
pw_sh = DataFrame(Date = sh_dew_hour.DateTime)
pw_sh.PW = dewpoint_to_PW.(sh_dew_hour.Dew)

pw_max_sh = PW_max(pw_sh.PW, pw_sh.Date)[2:7, :]                  # PW max for months of interest
pw_100_sh = PW_return_period(pw_sh.PW, pw_sh.Date, 100)[2:7, :]   # PW 100 for months of interest

Row,Month,PW_rp
,Int64,Float64
1,5,63.1963
2,6,85.3272
3,7,87.8987
4,8,90.2809
5,9,80.6228
6,10,58.7448


#### PMP

In [115]:
pmp_max_sh = PMP_mm(storm_sh.Rain, pw_storm_sh.PW, storm_sh.Date, pw_max_sh.PW_max)

435.5390728476822

In [116]:
pmp_100_sh = PMP_mm(storm_sh.Rain, pw_storm_sh.PW, storm_sh.Date, pw_100_sh.PW_rp)

427.2693208985716

## Hershfield K=15

In [18]:
sh_pmp_hershfield = PMP_Hershfield(data_sh.Rain, data_sh.Date, 15)

321.745147960414